In [1]:
import pandas as pd
import numpy as np
import openpyxl as opxl
import os.path
import pycountry
import pycountry_convert as pc
import datetime as dt
import json
import os.path

In [2]:
def importExcelSheets():
    magento = pd.read_excel('database_sheets/magento_may.xlsx')
    new_magento = pd.read_excel('database_sheets/magento_july.xlsx')
    magento_sept = pd.read_excel('database_sheets/magento_sept.xlsx')
    lot_master = pd.read_excel('database_sheets/lot_master.xlsx', dtype = object)
    prms = pd.read_excel('database_sheets/prms.xlsx')
    unspsc_codes = pd.read_excel('database_sheets/unspsc_codes.xlsx')
    unspsc_codes.columns = unspsc_codes.iloc[0]
    origin = pd.read_excel('database_sheets/country_of_origin.xlsx', dtype=object)
    pd.set_option("max_rows", None)
    pd.set_option("max_columns", None)
    return [magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept]

In [3]:
def tidyDescription(desc):
    desc = desc.replace('&TRADE', '')
    desc = desc.replace('Â®', '')
    desc = desc.replace('Î¼', '')
    desc = desc.replace('& Growâ„¢', '')
    desc = desc.replace('â‰', '')
    desc = desc.replace('&Beta', '')
    desc = desc.replace('Â‰¥', '')
    desc = desc.replace('Â', '')
    desc = desc.replace('<em>', '')
    desc = desc.replace('</em>', '')
    desc = desc.replace('®','')
    desc = desc.replace('<F128>','')
    desc = desc.replace('<130>','')
    desc = desc.replace('<F255>','')
    desc = desc.replace('&deg;', '°')
    desc = desc.replace('deg ', '°')
    if desc.endswith(','):
        desc = desc[:len(desc)-1]
    return desc

In [4]:
database_sheets = importExcelSheets()

In [14]:
def fillThomas_Old(filename, magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept):
    thomas = pd.read_excel('forms/thomas_form.xlsx', dtype = object)
    # thomas.columns = np.arange(len(thomas.columns))
    thomas.columns = thomas.iloc[3]
    wb = opxl.load_workbook(filename)
    # wb = opxl.load_workbook('Output for Thomas form.xlsx')
    # skus = wb['Fisher']
    skus = wb.active
    for i in range(2, skus.max_row+1):
        thomas.loc[i+19, 1] = str(skus['A'+str(i)].value)
        
    for i in range(21, skus.max_row+20):
        sku = thomas[1][i]
        product_info = magento.loc[magento['sku'] == sku]
        lot_info = lot_master.loc[lot_master['Product number'] == sku]
        origin_info = origin.loc[origin['Product number'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]
        unspsc_info = unspsc_codes.loc[unspsc_codes['Part Number'] == sku]
        product_info_july = new_magento.loc[new_magento['sku'] == sku]
        product_info_sept = magento_sept.loc[magento_sept['sku'] == sku]

        if not product_info_july.empty:
            tariff_code = product_info_july['tariff_code'].values[0]
        else:
            tariff_code = ''

        if not product_info.empty:
            if not product_info_sept.empty:
                pack_size_joined = product_info_sept['pack_size_joined'].values[0]
                product_type = product_info_july['product_type'].values[0]
                description = product_info_sept['description'].values[0]
                clone_name = product_info_sept['clone_name'].values[0]
                conjugate = product_info_sept['conjugate'].values[0]
                host = product_info_sept['host'].values[0]
                isotype = product_info_sept['isotype'].values[0]
                solubility = product_info_sept['solubility'].values[0]
                source = product_info_sept['source'].values[0]
                species_reactivity = product_info_sept['species_reactivity'].values[0]
                immunogen = product_info_sept['immunogen'].values[0]
                concentration = product_info_sept['concentration'].values[0]
                specificity = product_info_sept['specificity'].values[0]
                sterility = product_info_sept['sterility'].values[0]
                components = product_info_sept['components'].values[0]
                format_value = product_info_sept['format'].values[0]
                name = product_info_sept['name'].values[0]

            else:
                pack_size_joined = product_info['pack_size_joined'].values[0]
                product_type = product_info['product_type'].values[0]
                description = product_info['description'].values[0]
                clone_name = ''
                conjugate = ''
                host = ''
                isotype = ''
                solubility = ''
                source = ''
                species_reactivity = ''
                immunogen = ''
                concentration = product_info['concentration'].values[0]
                specificity = ''
                sterility = ''
                components = ''
                format_value = ''
                name = ''

            if not prms_info.empty:
                price = prms_info['USD List Price'].values[0]
                un_num = prms_info['UN#'].values[0]
                packing_group = prms_info['Packing Group'].values[0]
                storage_temp = prms_info['Storage Temp'].values[0]
                ship_temp = prms_info['Ship Temp'].values[0]
    #             country_of_origin = prms_info['Country of Origin (most recent lot)'].values[0]
            else:
                price = product_info['price'].values[0]
                packing_group = product_info['packing_group'].values[0]
                ship_temp = ''
                storage_temp = ''
    #             if not lot_info.empty:
    #                 country_of_origin = lot_info['Country of Origin'].values[0]
    #             else:
    #                 country_of_origin = ''
                    
            if not origin_info.empty:
                country_of_origin = origin_info.loc[origin_info['Expiration date -'] == max(origin_info['Expiration date -'].values)]['Country of Origin'].values[0]
            elif not prms_info.empty:
                country_of_origin = prms_info['Country of Origin (most recent lot)'].values[0]
            elif not lot_info.empty:
                country_of_origin = lot_info['Country of Origin'].values[0]
            else:
                country_of_origin = ''
                        
            if not unspsc_info.empty:
                unspsc = unspsc_info['UNSPSC'].values[0]
            else:
                unspsc = product_info['unspsc'].values[0]
            
            #name = product_info['name'].values[0].upper()
            # description = product_info['description'].values[0]
            short_desc = product_info['short_description'].values[0]
            img_link = product_info['base_image'].values[0]
            #price = product_info['price'].values[0]
            pack_size = product_info['pack_size_numeric_value'].values[0]
            unit = product_info['pack_size_unit_of_measure'].values[0]
            quantity = product_info['lk_packaging_facet'].values[0]
            #packing_group = product_info['packing_group'].values[0]
            #ship_temp = product_info['ship_conditions'].values[0]
            host = product_info['host'].values[0]
            # tariff_code = product_info['tariff_code'].values[0]
            msds_avail = skus['B'+str(i-1)].value
            cas_number = product_info['cas_number'].values[0]
            DOT_PSN = 'N/A'
            hazard_class = product_info['hazard_class'].values[0]
            biochem_physiol_actions = product_info['biochem_physiol_actions'].values[0]
            storage_and_handling = product_info['storage_and_handling'].values[0]
            group_name = product_info['prms_group_name'].values[0]
            #unspsc = product_info['unspsc'].values[0]
            img_link = product_info['base_image'].values[0]
            hazard_statements = product_info['hazard_statements'].values[0]
            keywords = product_info['meta_keywords'].values[0]
            # storage_temp = product_info['storage_and_handling'].values[0]
            grade = product_info['grade'].values[0]
            # pack_size_joined = product_info['pack_size_joined'].values[0]
            ph = product_info['ph'].values[0]
            weight_in_lb = new_magento.loc[new_magento['sku'] == sku]['weight'].values[0]
            purity = product_info['purity'].values[0]
            molecular_weight = product_info['molecular_weight'].values[0]
            key_applications = product_info['key_applications'].values[0]
            
    #         if type(quantity) == str:
    #             quantity = int(quantity.split('x')[0])
    #         weight_in_lb = convertWeightToPounds(pack_size, unit, quantity)
            
            if type(name) == str:
                name = tidyDescription(name)
                if (not (str(pack_size) + str(unit)) in name) and not ((str(pack_size_joined) in name)):
                    if len(name) > 40-len(str(pack_size_joined)):
                        name = name[:39-len(pack_size_joined)] + ' ' + pack_size_joined
                    else:
                        name = name + ' ' + str(pack_size_joined)
                if len(name) > 40:
                    name = name[:40]
                thomas[2][i] = name
            
            thomas[3][i] = 'EA'
            thomas[4][i] = 1
            
            if type(price) != str:
                if str(sku).startswith('11'):
                    thomas[5][i] = 0.89*price
                else:
                    thomas[5][i] = 0.8*price
                thomas[6][i] = price
            
            if ship_temp == 'DI':
                thomas[13][i] = 'Ice/Dry Ice'
                thomas[14][i] = '$10.00'
                thomas[15][i] = 'ITEM'
                thomas[31][i] = 'DRY ICE'
            elif ship_temp == 'CP':
                thomas[31][i] = 'ICE'
            elif ship_temp == 'AM':
                thomas[31][i] = 'RT'
            
            if type(cas_number) == str and len(cas_number) > 0:
                thomas[17][i] = 'Chemicals'
                thomas[52][i] = grade
                # thomas[53][i] = pack_size_joined
                thomas[54][i] = 'Bottle'
                thomas[55][i] = cas_number
                thomas[59][i] = ph
                
            thomas[21][i] = 'Y'
            thomas[22][i] = 'NONE'
            thomas[24][i] = country_of_origin
            
            tariff_code = str(tariff_code)
            if len(tariff_code) >= 4:
                thomas[23][i] = tariff_code
            
            if weight_in_lb < 0.5:
                thomas[25][i] = 7
                thomas[26][i] = 4
                thomas[27][i] = 5
                thomas[28][i] = 0.081
            elif weight_in_lb <= 1:
                thomas[25][i] = 12
                thomas[26][i] = 7
                thomas[27][i] = 5
                thomas[28][i] = 0.243
            else:
                thomas[25][i] = 12
                thomas[26][i] = 12
                thomas[27][i] = 12
                thomas[28][i] = 1
                
            thomas[29][i] = weight_in_lb
            
            if type(storage_temp) == str:
                if storage_temp == 'AM':
                    thomas[32][i] = 'RT'
                    thomas[33][i] = 'N/A'
                elif storage_temp == 'FR':
                    thomas[32][i] = '-20°C'
                    thomas[33][i] = 'Freezer'
                elif storage_temp == '70' or storage_temp == '80':
                    thomas[32][i] = '-80°C'
                    thomas[33][i] = 'Freezer'
                elif storage_temp == 'RF':
                    thomas[32][i] = '4°C'
                    thomas[33][i] = 'Refrigerator'
                # else:
                #     thomas[32][i] = storage_temp
            #thomas[32][i] = storage_temp
            
            if not lot_info.empty:
                creation_date = np.datetime64(lot_info['Creation date -'].values[0])
                expiration_date = np.datetime64(lot_info['Expiration date -'].values[0])
                shelf_life = expiration_date - creation_date
                shelf_life = shelf_life.astype('timedelta64[D]')/np.timedelta64(1, 'D')
                if shelf_life > 0:
                    thomas[34][i] = str(shelf_life) + ' days'
            
            thomas[42][i] = 'D'
            thomas[44][i] = 'No'
            thomas[53][i] = pack_size_joined
            
            if type(name) == str:
                name = tidyDescription(name)
                if product_type == 'configurable':
                    thomas[64][i] = name + ' ' + str(pack_size_joined)
                else:
                    thomas[64][i] = name
            
            thomas[65][i] = 'N/A'

            if type(description) == str and len(description) > 0:
                thomas[66][i] = description
            elif not product_info_sept.empty:
                thomas[66][i] = product_info_sept['application_notes'].values[0]
            
            specifications = ''
            if type(purity) == str and len(purity) > 0:
                specifications = specifications + 'Purity: ' + purity.replace('â‰¥', '')
            if type(molecular_weight) == str and len(molecular_weight) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Molecular Weight: ' + molecular_weight
            if type(key_applications) == str and len(key_applications) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Key Applications: ' + key_applications
            if type(concentration) == str and len(concentration) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Concentration: ' + concentration
            if type(clone_name) == str and len(clone_name) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Clone Name: ' + clone_name
            if type(conjugate) == str and len(conjugate) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Conjugate: ' + conjugate
            if type(host) == str and len(host) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Host: ' + host
            if type(isotype) == str and len(isotype) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Isotype: ' + isotype
            if type(solubility) == str and len(solubility) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Solubility: ' + solubility
            if type(source) == str and len(source) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Source: ' + source
            if type(species_reactivity) == str and len(species_reactivity) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Species Reactivity: ' + species_reactivity
            if type(immunogen) == str and len(immunogen) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Immunogen: ' + immunogen
            if type(specificity) == str and len(specificity) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Specificity: ' + specificity
            if type(sterility) == str and len(sterility) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Sterility: ' + sterility
            if type(components) == str and len(components) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Components: ' + components
            if type(format_value) == str and len(format_value) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Format: ' + format_value
            
            thomas[67][i] = specifications
            
            if type(keywords) == str:
                thomas[68][i] = keywords.replace(',', ' ')
            
            # thomas[69][i] = img_link
            
    new_thomas = opxl.load_workbook('forms/thomas_form.xlsx')
    thomas_sheet = new_thomas.active
    if thomas_sheet.max_row < skus.max_row:
        for j in range(skus.max_row - thomas_sheet.max_row):
            thomas_sheet.insert_rows(thomas_sheet.max_row-1)
    i = 1
    for row in thomas_sheet.iter_rows(min_row=23):
        if i < skus.max_row:
            for j in range(len(thomas.columns)):
                row[j].value = thomas[j][i+2]
        else:
            break
        i = i+1

    # new_thomas.save('../../outputs/old_product_outputs/old_thomas_output.xlsx')
    new_thomas.save('outputs/old_product_outputs/old_thomas_output.xlsx')

In [8]:
pd.set_option('max_rows', None)

In [13]:
thomas = pd.read_excel('forms/thomas_form.xlsx', dtype=object)
thomas.columns = thomas.iloc[3]
thomas.iloc[0:5 , :]

3,THOMAS #\n(Leave blank),MFR. NUMBER,INVENTORY DESCRIPTION\n(40 character maximum including spaces)\nMUST include voltage of equipment\nand/or package size to clarify differences amongst products. \nWE CAN NOT ACCEPT DUPLICATE DESCRIPTIONS FOR DIFFERENT PRODUCTS - see Examples Tab,PURCHASE UNIT OF MEASURE\n(EA PK CS ETC.),SIZE QTY\nNumber needed Only,PUoM COST,PUoM LIST,DISCOUNT %,2ND PUoM\n(PK CS ETC.),2ND PUoM SIZE QTY,2ND PUoM Cost\n,2ND PUoM List\n,NaN,Any Additional Fees (NOT included in PUoM COST),NaN,NaN,NaN,"Section\n(Supplies,\nInstruments,\nEquipment,\nChemicals)",CATEGORY\n(e.g. Balances),Sub Category/Type\n(e.g. Analytical)\n(optional),NaN,DROP SHIP?\n(Y or N),Shipping or Drop minimums or penalties?\nIf yes please state,HARMONIZATION CODE,COUNTRY OF ORIGIN\n(Need only 1),SHIPPING SIZE in INCHES or CUBIC FEET,NaN,NaN,NaN,SHIP WEIGHT,NaN,SHIPPING CONDITIONS \n(RT ICE DRY ICE),STORAGE CONDITIONS \n(RT 4°C -20°C -80°C),Refrigeration Requirements\n(Refrigerator or Freezer),SHELF LIFE,HAZARDOUS MATERIAL \n(Y or N),UN /NA # \n(if hazardous),FOR CHEMICALS,NaN,NaN,NaN,CHEMICAL CODE \n(C T R or Y),MSDS CODE \n( A B C OR D),NaN,REPLACES PART#\n(if applicable enter PT# being replaced),FISHER #,VWR #,COLE PARMER #,DAIGGER #,OTHER COMPETITOR #1,OTHER COMPETITOR #2,NaN,Chemical Attributes for Website,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WEBSITE DESCRIPTION\n\nThis is a simple line listing description.\ninclude voltage of equipment \nWE CAN NOT ACCEPT DUPLICATE DESCRIPTIONS FOR DIFFERENT PRODUCTS - see Examples Tab,Addition to Existing Product Group?,COPY or Copy Address,Specifications,KEYWORDS\nList in single line (e.g.: bottle narrow mouth HDPE) \nHelps in the search of your products on the website,IMAGE\n,Additional Information
0,Contact Email:,patricia.swanner@mpbio.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Contact Phone:,100-100-6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,NaN,NaN,NaN,NaN,NaN,NaN,REQUIRED (if Applicable),NaN,NaN,NaN,REQUIRED,REQUIRED,NaN,NaN,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REQUIRED if Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,NaN,NaN,REQUIRED if Applicable,NaN,REQUIRED,REQUIRED,REQUIRED,NaN,NaN,REQUIRED,NaN
3,THOMAS #\n(Leave blank),MFR. NUMBER,INVENTORY DESCRIPTION\n(40 character maximum i...,PURCHASE UNIT OF MEASURE\n(EA PK CS ETC.),SIZE QTY\nNumber needed Only,PUoM COST,PUoM LIST,DISCOUNT %,2ND PUoM\n(PK CS ETC.),2ND PUoM SIZE QTY,2ND PUoM Cost\n,2ND PUoM List\n,NaN,Any Additional Fees (NOT included in PUoM COST),NaN,NaN,NaN,"Section\n(Supplies,\nInstruments,\nEquipment,\...",CATEGORY\n(e.g. Balances),Sub Category/Type\n(e.g. Analytical)\n(optional),NaN,DROP SHIP?\n(Y or N),Shipping or Drop minimums or penalties?\nIf ye...,HARMONIZATION CODE,COUNTRY OF ORIGIN\n(Need only 1),SHIPPING SIZE in INCHES or CUBIC FEET,NaN,NaN,NaN,SHIP WEIGHT,NaN,SHIPPING CONDITIONS \n(RT ICE DRY ICE),STORAGE CONDITIONS \n(RT 4°C -20°C -80°C),Refrigeration Requirements\n(Refrigerator or F...,SHELF LIFE,HAZARDOUS MATERIAL \n(Y or N),UN /NA # \n(if hazardous),FOR CHEMICALS,NaN,NaN,NaN,CHEMICAL CODE \n(

In [45]:
print(len(vwr.columns))
# vwr.iloc[1, 179]

180


' DEA Establishment Registration Number'

In [15]:
vwr_filled = fillVWR_Old('input_sheets/VWR testing.xlsx', database_sheets[0], database_sheets[1], database_sheets[2], database_sheets[3], database_sheets[4], database_sheets[5])

In [17]:
display(vwr_filled)

1,NaN,Supplier Name,Email Address for Contact,Contact Phone,Vendor Part Number,40 Character Description,Enhanced Description,Purchase Unit of Measure,Minimum Order?,Vendor Price,Vendor Currency,Future Vendor Price,Future Pricing Effectivity Date,US Item Category Group,CA Item Category Group,lot controlled,Mandatory Expiration Date for lot control,Component Quantity,Component Measure,Component Size,Selling UM1,UM1 List Price,Future UM1 List Price,Selling UM2,UM2 Qty,UM2 List Price,Future UM2 List Price,Selling UM3,UM3 Qty,UM3 List Price,Future UM3 List Price,Gross Margin 1,Gross Margin 2,Gross Margin 3,Policy Code,Shield,Stocking Sales Cost,Future Stocking Sales Cost,Product Code,UM1 Weight,UM1 Length,UM1 Width,UM1 Height,UM1 Ship as Is?,UM2 Weight,UM2 Length,UM2 Width,UM2 Height,UM2 Ship as Is?,UM3 Weight,UM3 Length,UM3 Width,UM3 Height,UM3 Ship as Is?,US Harmonization Code,CA Harmonization Code,HTS & ECCN Ruling # (if applicable),ECCN/US MUL Cat. #,Country of Origin,Eligible for NAFTA,Other US or CA Free Trade Agreements (FTAs),Animal/Plant or Synthetic,If Animal Origin - Confirm Genus/Species & CITES compliance,Sustainability Info,OGA/OGD (Other Government Agencies/Departments),MSDS available,WHMIS Compliant,DOT Proper Ship Name,"DOT - If NOS, Technical Name",DOT - UN Identification #,"DOT - Packing Group I, II, or III",Reportable Quanitity (RQ),Limited Quantity,DOT-SP,IATA Proper Ship Name,"IATA - If NOS, Technical Name ()",IATA - UN Identification #,"IATA - Packing Group I, II, or III",TDG Proper Ship Name,"TDG - If NOS, Technical Name ()",TDG - UN Identification #,"TDG - Packing Group I, II, or III",Type of Outside Container Used,Inner Bottles Pressure Tested?,Inner Bottle Material,CAS#,"If a Chemical, CWC?",Regulated by DEA or TTB?,Regulated by Health Canada as a Precursor Chemical?,Certificate of Analysis,Certificate of Sterility,Certificate of Quality/Conformance,Electrical Certification Code,Electrical w/Motors,Chemical Grade,FDA Compendial,FDA / HC Medical Device Class,510k/PMA # / HC License #,Import FDA Code,Import DEV (Foreign Registration Number),Import FDA LST Number,Rx labeled?,NDC#,Regulated by Health Canada as a Drug or Natural Health Product?,Actual Manufacturer & full address,Drop Ship Fee?,Air Eligible?,Ship Temp Requirement,"If on ice, blue or dry?",Dated Shelf Life?,"If dated, returnable?",Storage Temp Requirement,Replenishment Supplier Number,Private Label?,"From Marketing Standpoint, US / CA Salability","From Regulatory Standpoint, US / CA Salability",Can Material be Exported?,Material Group,Detail Material Group,Hierarchy Code,Product Category,Material Type,Profit Center,VWR Part Number,MEL Number,PN Block Start,Initial Sell Hold Code,Initial Purchase Hold Code,Special Svc & Handling,Packing Code,SAP PID #,Transport Restrictions,Trans Temp Restric,AA,AO,AU,GO,GU,Regulated Prod (sales org 8000),Regulated Prod (sales org 8600),Med Dev/ Assess. No (sales org 8000),Med Dev/ Assess. No (sales org 8600),Lab/Office Code,Regulatory Field (sales org 8000),Regulatory Field (sales org 8600),Haz Storage Coding P1,Haz Storage Coding P2,Haz Storage Coding P3,AIRS Status,AIRS Import Details for Requirements,AIRS Version,AIRS ODG Extension,AIRS Origin,AIRS End Use,AIRS Comments/Requirements,AIRS Date,AIRS Classifier,Trade Approver Initials,Include on us.vwr.com,Include on ca.vwr.com,Include on wardsci.com,Include on wardsci.ca,Include on sargentwelch.com,Include on sargentwelch.ca,Children's Product?,Child's Toy?,CPSA testing reports / CofC,Cautionary Advertistement,Cautionary Labeling,Fish & Wildlife,Source of wildlife,Venomous wildlife?,USDA?,Permits and Restrictions,Zoosanitary Certificates,Radiation Emitting Devices,NRCan Regulated?,CFIA Regulated for CA import & AIRS codes,PHAC regulated for CA import?,DEA Establishment Registration Number
0,The buttons below should allow you to hide/sho...,Supplier Info,NaN,NaN,Description,NaN,NaN,Purchasing Info,NaN,NaN,NaN,NaN,NaN,Warehousing,NaN,NaN,NaN,Selling Info,NaN,NaN,